In [1]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 24C6-591C

 Directory of C:\Users\ninja\Desktop\takvim\11 hafta\HAFTA_1_18_06_2024\projeler\drone manevra\drone manevra\Grup-2

20.06.2024  15:26    <DIR>          .
20.06.2024  15:26    <DIR>          ..
20.06.2024  14:23    <DIR>          .ipynb_checkpoints
20.06.2024  14:34           668.136 best_model.h5
21.06.2022  12:09        12.460.113 drive-download-20220621T090805Z-001.zip
20.06.2024  15:26           235.411 drone maneuvers classification.ipynb
20.06.2024  14:17           668.232 drone_maneuver_classifier.h5
20.06.2024  14:16            43.221 drone_maneuver_features.csv
04.05.2022  13:20         1.634.334 Geri-v1.wav
04.05.2022  13:20         1.740.174 Geri-v2.wav
04.05.2022  13:20         1.702.070 Geri-v3.wav
04.05.2022  11:50         1.723.238 ileri-v1.wav
04.05.2022  11:50         1.879.878 ileri-v2.wav
04.05.2022  11:50         1.685.138 ileri-v3.wav
04.05.2022  13:20         1.723.238 Saga-v1.wav
04.05.2022  13:

In [2]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import librosa
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

C:\Users\ninja\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
data_dir = "C:/Users/ninja/Desktop/takvim/11 hafta/HAFTA_1_18_06_2024/projeler/drone manevra/drone manevra/Grup-2/"
audio_files = [f for f in os.listdir(data_dir) if f.endswith('.wav')]

In [4]:
audio_files

['Geri-v1.wav',
 'Geri-v2.wav',
 'Geri-v3.wav',
 'ileri-v1.wav',
 'ileri-v2.wav',
 'ileri-v3.wav',
 'Sag-v4.wav',
 'Saga-v1.wav',
 'Saga-v2.wav',
 'Saga-v3.wav',
 'Sola-v1.wav',
 'Sola-v2.wav',
 'Sola-v3.wav',
 'Sola-v5.wav']

# Data Augmentation + feature extraction functions

In [5]:
def pitch_shift(audio, sample_rate, n_steps):
    return librosa.effects.pitch_shift(audio, sample_rate, n_steps)

def time_stretch(audio, rate):
    return librosa.effects.time_stretch(audio, rate)

def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

features = []
labels = []

# reading sounds and feature extraction

In [7]:
for file_name in audio_files:
    file_path = os.path.join(data_dir, file_name)
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')

    # orijinal
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    features.append(mfccs_scaled)

    # çoğaltılmış
    pitch_shifted = pitch_shift(audio, sample_rate, n_steps=2)
    time_stretched = time_stretch(audio, rate=0.8)
    noise_added = add_noise(audio)

    mfccs_pitch_shifted = librosa.feature.mfcc(y=pitch_shifted, sr=sample_rate, n_mfcc=40)
    mfccs_time_stretched = librosa.feature.mfcc(y=time_stretched, sr=sample_rate, n_mfcc=40)
    mfccs_noise_added = librosa.feature.mfcc(y=noise_added, sr=sample_rate, n_mfcc=40)

    features.append(np.mean(mfccs_pitch_shifted.T, axis=0))
    features.append(np.mean(mfccs_time_stretched.T, axis=0))
    features.append(np.mean(mfccs_noise_added.T, axis=0))

    # Label
    if 'Geri' in file_name:
        label = 'geri'
    elif 'ileri' in file_name:
        label = 'ileri'
    elif 'Saga' in file_name or 'Sag' in file_name:
        label = 'sağa'
    elif 'Sola' in file_name:
        label = 'sola'
    
    labels.extend([label, label, label, label])

C:\Users\ninja\AppData\Local\Temp\ipykernel_27024\1917894006.py:2: FutureWarning: Pass sr=22050, n_steps=2 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.pitch_shift(audio, sample_rate, n_steps)
C:\Users\ninja\AppData\Local\Temp\ipykernel_27024\1917894006.py:5: FutureWarning: Pass rate=0.8 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.effects.time_stretch(audio, rate)


# MFCCS AND LABELS

In [8]:
features = np.array(features)
labels = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)


df = pd.DataFrame(features)
df['label'] = labels

In [9]:
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-285.793182,111.224174,-2.820960,15.090815,27.493719,30.877007,-0.027800,13.044175,-3.031512,16.967196,...,-4.079960,-1.756243,1.287041,-0.954990,0.357522,-0.943054,0.145208,-2.618156,-3.433697,geri
1,-316.461060,103.125267,-0.889450,19.395672,37.119801,19.913267,2.476149,9.578638,4.971298,13.985589,...,4.247362,-4.128846,-1.397539,-3.687644,-0.668049,-3.714141,0.565353,0.306265,1.829687,geri
2,-324.880768,109.550911,-3.006274,15.179991,27.528976,30.903843,0.209873,13.385638,-3.060223,16.961319,...,-3.410093,-1.575887,0.916751,-1.552660,-0.290086,-1.938637,-0.477753,-2.456151,-3.202936,geri
3,-207.322747,73.793810,7.498273,7.972365,25.440176,21.681496,5.506397,4.649271,3.376908,10.278045,...,-4.054845,-0.721953,0.758001,0.074818,-0.276411,0.076241,-0.171124,-1.903070,-3.012149,geri
4,-237.887222,119.698967,-6.027120,17.222549,29.685616,32.897388,-2.380576,11.849514,-3.522478,17.045530,...,0.586014,1.975310,1.688975,-1.262375,-0.275650,-2.697506,-2.625662,-4.455345,0.126582,geri
5,-271.381439,111.841560,-4.867367,21.401793,39.025574,21.094189,-0.880764,7.815508,3.539521,13.821593,...,2.116592,-5.961555,-0.733243,-3.848724,0.650723,-1.866790,2.335452,-1.463108,-2.566632,geri
6,-275.363831,117.434082,-5.711973,16.642897,29.434448,32.496693,-1.895764,11.541454,-3.801913,16.245113,...,-0.032407,1.688938,1.554442,-0.395243,0.220208,-2.161443,-2.768564,-4.332443,0.030065,geri
7,-184.046902,87.773336,6.306405,10.123757,30.308167,24.647579,4.397537,3.666950,3.594029,10.872073,...,-0.089891,2.499898,0.980190,-0.372308,-0.760663,-1.477311,-3.084585,-3.385463,-0.091470,geri
8,-282.686157,117.875717,-5.309691,15.825414,27.173628,30.836378,-0.774167,12.551248,-2.739361,15.990652,...,-2.048947,-0.003442,2.523432,-0.104449,1.874497,-1.196824,-1.000326,-2.484491,-3.481593,geri
9,-314.015778,108.613686,-3.514414,20.322048,35.808342,19.730074,1.308592,9.571907,4.103570,14.757357,...,3.985307,-4.085641,-0.580376,-4.214109,-1.951801,-3.340245,0.663711,-1.096935,1.804882,geri


In [10]:
features.shape

(56, 40)

In [11]:
labels_encoded


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

 
model = Sequential()
model.add(Dense(256, input_shape=(40,), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))  # 4 classes: geri, ileri, sağa, sola

 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
 
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint])
 
model.load_weights('best_model.h5')
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/500
1/2 [==============>...............] - ETA: 0s - loss: 13.8180 - accuracy: 0.4062
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to best_model.h5
2/2 [==============================] - 1s 164ms/step - loss: 13.1617 - accuracy: 0.4545 - val_loss: 11.2213 - val_accuracy: 0.2500
Epoch 2/500
1/2 [==============>...............] - ETA: 0s - loss: 15.6966 - accuracy: 0.3750
Epoch 2: val_accuracy did not improve from 0.25000
2/2 [==============================] - 0s 23ms/step - loss: 16.6174 - accuracy: 0.2955 - val_loss: 12.0314 - val_accuracy: 0.2500
Epoch 3/500
1/2 [==============>...............] - ETA: 0s - loss: 18.3617 - accuracy: 0.2188
Epoch 3: val_accuracy did not improve from 0.25000
2/2 [==============================] - 0s 24ms/step - loss: 19.7184 - accuracy: 0.1818 - val_loss: 10.0931 - val_accuracy: 0.2500
Epoch 4/500
1/2 [==============>...............] - ETA: 0s - loss: 12.0221 - accuracy: 0.3750
Epoch 4: val_accuracy did not improve from 0.250

2/2 [==============================] - 0s 23ms/step - loss: 3.3880 - accuracy: 0.2273 - val_loss: 1.1904 - val_accuracy: 0.4167
Epoch 31/500
1/2 [==============>...............] - ETA: 0s - loss: 3.7191 - accuracy: 0.2500
Epoch 31: val_accuracy improved from 0.50000 to 0.58333, saving model to best_model.h5
2/2 [==============================] - 0s 36ms/step - loss: 3.4418 - accuracy: 0.2955 - val_loss: 1.1534 - val_accuracy: 0.5833
Epoch 32/500
1/2 [==============>...............] - ETA: 0s - loss: 3.1485 - accuracy: 0.3750
Epoch 32: val_accuracy did not improve from 0.58333
2/2 [==============================] - 0s 21ms/step - loss: 3.1228 - accuracy: 0.3636 - val_loss: 1.1268 - val_accuracy: 0.5833
Epoch 33/500
1/2 [==============>...............] - ETA: 0s - loss: 2.7901 - accuracy: 0.3125
Epoch 33: val_accuracy did not improve from 0.58333
2/2 [==============================] - 0s 21ms/step - loss: 2.6314 - accuracy: 0.3409 - val_loss: 1.1019 - val_accuracy: 0.5833
Epoch 34/500
1/

1/2 [==============>...............] - ETA: 0s - loss: 1.6230 - accuracy: 0.4375
Epoch 60: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 21ms/step - loss: 1.7235 - accuracy: 0.4091 - val_loss: 1.0699 - val_accuracy: 0.5000
Epoch 61/500
1/2 [==============>...............] - ETA: 0s - loss: 1.3418 - accuracy: 0.5312
Epoch 61: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 21ms/step - loss: 1.4634 - accuracy: 0.5000 - val_loss: 1.0871 - val_accuracy: 0.5000
Epoch 62/500
1/2 [==============>...............] - ETA: 0s - loss: 1.8225 - accuracy: 0.3125
Epoch 62: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 21ms/step - loss: 2.0097 - accuracy: 0.2955 - val_loss: 1.0920 - val_accuracy: 0.5000
Epoch 63/500
1/2 [==============>...............] - ETA: 0s - loss: 1.8751 - accuracy: 0.5312
Epoch 63: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 22m

Epoch 90/500
1/2 [==============>...............] - ETA: 0s - loss: 1.7029 - accuracy: 0.4062
Epoch 90: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 21ms/step - loss: 1.5256 - accuracy: 0.4545 - val_loss: 0.9070 - val_accuracy: 0.5833
Epoch 91/500
1/2 [==============>...............] - ETA: 0s - loss: 1.3972 - accuracy: 0.3438
Epoch 91: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 20ms/step - loss: 1.6012 - accuracy: 0.3182 - val_loss: 0.8923 - val_accuracy: 0.5833
Epoch 92/500
1/2 [==============>...............] - ETA: 0s - loss: 1.8504 - accuracy: 0.3438
Epoch 92: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 20ms/step - loss: 1.7002 - accuracy: 0.3636 - val_loss: 0.8804 - val_accuracy: 0.5833
Epoch 93/500
1/2 [==============>...............] - ETA: 0s - loss: 1.5290 - accuracy: 0.3438
Epoch 93: val_accuracy did not improve from 0.75000
2/2 [===========================

Epoch 120/500
1/2 [==============>...............] - ETA: 0s - loss: 1.0837 - accuracy: 0.6250
Epoch 120: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 22ms/step - loss: 1.0866 - accuracy: 0.5682 - val_loss: 0.8906 - val_accuracy: 0.6667
Epoch 121/500
1/2 [==============>...............] - ETA: 0s - loss: 1.0115 - accuracy: 0.5938
Epoch 121: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 21ms/step - loss: 1.0147 - accuracy: 0.5909 - val_loss: 0.8945 - val_accuracy: 0.6667
Epoch 122/500
1/2 [==============>...............] - ETA: 0s - loss: 1.2684 - accuracy: 0.4688
Epoch 122: val_accuracy did not improve from 0.75000
2/2 [==============================] - 0s 22ms/step - loss: 1.1544 - accuracy: 0.5227 - val_loss: 0.8989 - val_accuracy: 0.5833
Epoch 123/500
1/2 [==============>...............] - ETA: 0s - loss: 1.2487 - accuracy: 0.4062
Epoch 123: val_accuracy did not improve from 0.75000
2/2 [===================

...

Epoch 360/500
1/2 [==============>...............] - ETA: 0s - loss: 0.2618 - accuracy: 0.9375
Epoch 360: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 22ms/step - loss: 0.3775 - accuracy: 0.8636 - val_loss: 0.1896 - val_accuracy: 0.9167
Epoch 361/500
1/2 [==============>...............] - ETA: 0s - loss: 0.5194 - accuracy: 0.8438
Epoch 361: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 21ms/step - loss: 0.4355 - accuracy: 0.8864 - val_loss: 0.1890 - val_accuracy: 0.9167
Epoch 362/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3319 - accuracy: 0.8438
Epoch 362: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 21ms/step - loss: 0.3025 - accuracy: 0.8864 - val_loss: 0.1924 - val_accuracy: 0.9167
Epoch 363/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3478 - accuracy: 0.8750
Epoch 363: val_accuracy did not improve from 1.00000
2/2 [===================

Epoch 390/500
1/2 [==============>...............] - ETA: 0s - loss: 0.5661 - accuracy: 0.7188
Epoch 390: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 25ms/step - loss: 0.6008 - accuracy: 0.7500 - val_loss: 0.1702 - val_accuracy: 1.0000
Epoch 391/500
1/2 [==============>...............] - ETA: 0s - loss: 0.4376 - accuracy: 0.7500
Epoch 391: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 26ms/step - loss: 0.3759 - accuracy: 0.8182 - val_loss: 0.1752 - val_accuracy: 1.0000
Epoch 392/500
1/2 [==============>...............] - ETA: 0s - loss: 0.4472 - accuracy: 0.7500
Epoch 392: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 25ms/step - loss: 0.3388 - accuracy: 0.8182 - val_loss: 0.1816 - val_accuracy: 1.0000
Epoch 393/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3711 - accuracy: 0.8438
Epoch 393: val_accuracy did not improve from 1.00000
2/2 [===================

Epoch 420/500
1/2 [==============>...............] - ETA: 0s - loss: 0.1607 - accuracy: 0.9688
Epoch 420: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 24ms/step - loss: 0.2927 - accuracy: 0.9091 - val_loss: 0.1166 - val_accuracy: 1.0000
Epoch 421/500
1/2 [==============>...............] - ETA: 0s - loss: 0.4306 - accuracy: 0.8125
Epoch 421: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 23ms/step - loss: 0.3480 - accuracy: 0.8636 - val_loss: 0.1164 - val_accuracy: 1.0000
Epoch 422/500
1/2 [==============>...............] - ETA: 0s - loss: 0.2541 - accuracy: 0.9375
Epoch 422: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 22ms/step - loss: 0.2373 - accuracy: 0.9545 - val_loss: 0.1121 - val_accuracy: 1.0000
Epoch 423/500
1/2 [==============>...............] - ETA: 0s - loss: 0.1381 - accuracy: 1.0000
Epoch 423: val_accuracy did not improve from 1.00000
2/2 [===================

Epoch 450/500
1/2 [==============>...............] - ETA: 0s - loss: 0.1713 - accuracy: 0.9062
Epoch 450: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 22ms/step - loss: 0.2304 - accuracy: 0.8864 - val_loss: 0.0828 - val_accuracy: 1.0000
Epoch 451/500
1/2 [==============>...............] - ETA: 0s - loss: 0.2611 - accuracy: 0.9375
Epoch 451: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 22ms/step - loss: 0.2576 - accuracy: 0.9091 - val_loss: 0.0792 - val_accuracy: 1.0000
Epoch 452/500
1/2 [==============>...............] - ETA: 0s - loss: 0.2557 - accuracy: 0.8750
Epoch 452: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 23ms/step - loss: 0.2012 - accuracy: 0.9091 - val_loss: 0.0733 - val_accuracy: 1.0000
Epoch 453/500
1/2 [==============>...............] - ETA: 0s - loss: 0.1746 - accuracy: 0.9062
Epoch 453: val_accuracy did not improve from 1.00000
2/2 [===================

Epoch 480/500
1/2 [==============>...............] - ETA: 0s - loss: 0.2109 - accuracy: 0.9062
Epoch 480: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 23ms/step - loss: 0.2061 - accuracy: 0.9091 - val_loss: 0.0274 - val_accuracy: 1.0000
Epoch 481/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3514 - accuracy: 0.8125
Epoch 481: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 22ms/step - loss: 0.3144 - accuracy: 0.8409 - val_loss: 0.0323 - val_accuracy: 1.0000
Epoch 482/500
1/2 [==============>...............] - ETA: 0s - loss: 0.1089 - accuracy: 1.0000
Epoch 482: val_accuracy did not improve from 1.00000
2/2 [==============================] - 0s 23ms/step - loss: 0.1016 - accuracy: 1.0000 - val_loss: 0.0397 - val_accuracy: 1.0000
Epoch 483/500
1/2 [==============>...............] - ETA: 0s - loss: 0.1893 - accuracy: 0.9062
Epoch 483: val_accuracy did not improve from 1.00000
2/2 [===================